In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

import configparser
sys.path.insert(0, 'Utils/')

from AISDataManager import AISDataManager
import SimpleUtils as sU
import Constants as c
import TimeUtils as timeUtils
import datetime

aISDM = AISDataManager()
trainData = 0.8
testData = 1 - trainData
print("Train data =", trainData)
print("Test data =", testData)

Train data = 0.8
Test data = 0.19999999999999996


In [3]:
srcFile = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/TrackLen_1004.csv"
trajDF, _ = aISDM.load_data_from_csv(srcFile)
print(trajDF)
testDataPoints = int(trajDF.shape[0]*testData)
print(testDataPoints)

            MMSI  TRAJ_NUM    TRAJ_TIME   SOG_MEAN  SOG_MEDIAN   SOG_STD  \
0      355745000         0   725.950000  15.160903       16.10  1.931336   
1      355745000         1  1296.166667   8.899553        9.00  0.857530   
2      355745000         2   705.400000  15.576110       16.10  1.462801   
3      355745000         3   826.450000  15.576268       16.40  1.875382   
4      355861000         0   258.500000  17.223502       17.20  0.214197   
5      355861000         1   239.466667  10.875120       10.60  2.064478   
6      355861000         2   536.016667  16.176327       16.85  2.012012   
7      477655900         0   118.000000   9.502857        9.60  0.232651   
8      477655900         1   514.833333   9.919182        9.90  0.195502   
9      477655900         2   493.316667  10.287204       10.30  0.263319   
10     477655900         3   149.500000  10.334615       10.30  0.117276   
11     477655900         4   138.816667  10.123423       11.60  3.137803   
12     47765

In [4]:
trajDataAtLeast300Minutes = trajDF[trajDF['TRAJ_TIME'] > 300]
trajDataAtLeast300Minutes = trajDataAtLeast300Minutes.reset_index(drop=True)

In [5]:
random.seed(0)
testIdx = random.sample(range(0, trajDataAtLeast300Minutes.shape[0]), testDataPoints)
print(testIdx)

[13835, 6311, 12418, 6890, 663, 4242, 8376, 7961, 6634, 12841, 13596, 4969, 7808, 5866, 9558, 3578, 8268, 2281, 4617, 2289, 12383, 1553, 10131, 13097, 4104, 8725, 11553, 13274, 9861, 2407, 5081, 1618, 11957, 1208, 13934, 11206, 5409, 7735, 9171, 1649, 5796, 7113, 5180, 10008, 10492, 3350, 9052, 7815, 7253, 14177, 8541, 4267, 1020, 13189, 8989, 230, 1528, 11791, 13764, 6534, 11636, 13514, 12862, 10947, 10244, 18, 10025, 8086, 13566, 14214, 5458, 3996, 11964, 5328, 11528, 1031, 3130, 9298, 3632, 3909, 13162, 2334, 13159, 8896, 7339, 1494, 1318, 5243, 8322, 8016, 1786, 4938, 9031, 4769, 11576, 2044, 8969, 5451, 13344, 8852, 3329, 14302, 9882, 8965, 9627, 4712, 7290, 1501, 9769, 13073, 6306, 5194, 9431, 3966, 4756, 3012, 3102, 13458, 3059, 540, 10039, 10758, 4260, 7807, 1131, 1471, 11120, 12412, 2133, 2450, 633, 13802, 1314, 11457, 13588, 8857, 11198, 6410, 13727, 11555, 8594, 4515, 8549, 13297, 3858, 13917, 3525, 11132, 9663, 13521, 6871, 9497, 4509, 7382, 8071, 10817, 10505, 11472, 12997

In [6]:
#contains list of trajectories used for testing
trajDFTest = trajDataAtLeast300Minutes.iloc[testIdx,:].reset_index(drop=True)
print(trajDFTest)

           MMSI  TRAJ_NUM    TRAJ_TIME   SOG_MEAN  SOG_MEDIAN   SOG_STD  \
0     563722000         3   858.716667  12.538992        16.1  4.470210   
1     636015743         2   612.533333  18.175936        19.2  2.299396   
2     354839000        22   869.383333  13.616941        14.8  3.262880   
3     636016819         1   873.633333  12.613297        12.7  0.497944   
4     357405000        17  1220.833333  11.010160        10.1  1.393167   
5     477133700         3  1848.500000   8.706382         8.9  1.313365   
6     319570000         4  1051.950000  10.411534         9.4  2.728419   
7     565819000        17  1176.600000  12.061364        11.3  2.462064   
8     235009240        25   866.316667  16.506305        17.0  1.547386   
9     353025000         6   897.983333  16.031539        20.1  5.154630   
10    565048000         8   811.516667  12.888292        13.0  1.130548   
11    413161000         6  1085.350000  11.213426        11.6  1.810692   
12    566922000         8

In [7]:
destFileTest = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1004_Test.csv"
print(destFileTest)
aISDM.save_data_to_csv(trajDFTest, destFileTest)

/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1004_Test.csv


0

In [8]:
print(trajDFTest.shape)

(3550, 8)


In [12]:
trainIdx = []
for iDX in range(trajDF.shape[0]):
    tempTrajInfo = trajDF.iloc[iDX]
    existInTest = 0
    if(tempTrajInfo['TRAJ_TIME'] > 90):
        for testIDX in range(trajDFTest.shape[0]):
            tempTestTrajInfo = trajDFTest.iloc[testIDX]
            
            if(tempTrajInfo['MMSI'] == tempTestTrajInfo['MMSI']) \
            and (tempTrajInfo['TRAJ_NUM'] == tempTestTrajInfo['TRAJ_NUM']):
                existInTest = 1
                break
        if(existInTest == 0):
            trainIdx.append(iDX)

In [13]:
len(trainIdx)

14196

In [14]:
#contains list of trajectories used for training
trajDFTrain = trajDF.iloc[trainIdx,:].reset_index(drop=True)
print(trajDFTrain)

            MMSI  TRAJ_NUM    TRAJ_TIME   SOG_MEAN  SOG_MEDIAN   SOG_STD  \
0      355745000         0   725.950000  15.160903        16.1  1.931336   
1      355745000         2   705.400000  15.576110        16.1  1.462801   
2      355861000         0   258.500000  17.223502        17.2  0.214197   
3      355861000         1   239.466667  10.875120        10.6  2.064478   
4      477655900         0   118.000000   9.502857         9.6  0.232651   
5      477655900         1   514.833333   9.919182         9.9  0.195502   
6      477655900         3   149.500000  10.334615        10.3  0.117276   
7      477655900         4   138.816667  10.123423        11.6  3.137803   
8      477655900         7   947.133333  11.019112        11.4  1.112152   
9      477655900         8   815.366667  16.820000        19.5  4.380213   
10     477655900        10   740.433333  18.219938        19.3  2.761984   
11     477655900        11  1299.683333  10.684246        10.6  0.750603   
12     47765

In [15]:
destFileTrain = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1004_Train.csv"
print(destFileTrain)
aISDM.save_data_to_csv(trajDFTrain, destFileTrain)

/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1004_Train.csv


0

In [16]:
srcFileTanker = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/TrackLen_1024.csv"
trajDFTank, _ = aISDM.load_data_from_csv(srcFileTanker)
print(trajDFTank)
testDataPointsTanker = int(trajDFTank.shape[0]*testData)
print(testDataPointsTanker)
trajDataAtLeast300MinutesTanker = trajDFTank[trajDFTank['TRAJ_TIME'] > 300]
trajDataAtLeast300MinutesTanker = trajDataAtLeast300MinutesTanker.reset_index(drop=True)
random.seed(0)
testIdxTanker = random.sample(range(0, trajDataAtLeast300MinutesTanker.shape[0]), testDataPointsTanker)
print(testIdxTanker)
#contains list of trajectories used for testing
trajDFTestTanker = trajDataAtLeast300MinutesTanker.iloc[testIdxTanker,:].reset_index(drop=True)
print(trajDFTestTanker)
destFileTestTanker = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1024_Test.csv"
print(destFileTestTanker)
aISDM.save_data_to_csv(trajDFTestTanker, destFileTestTanker)
print(trajDFTestTanker.shape)

           MMSI  TRAJ_NUM    TRAJ_TIME   SOG_MEAN  SOG_MEDIAN   SOG_STD  \
0     241132000         0   410.516667  11.074719       11.10  0.331163   
1     241132000         1  1108.833333  11.416632       11.50  0.339299   
2     241132000         2  1099.183333  11.668898       11.60  0.364481   
3     241132000         3   254.500000  11.174439       11.00  0.415598   
4     241132000         4   429.850000  10.641176       10.70  0.285981   
5     241132000         5   950.150000  12.039781       12.10  0.761009   
6     241132000         6  1076.333333  11.917368       12.00  0.640760   
7     241132000         7   439.983333  11.857743       12.00  0.521863   
8     241132000         8   984.683333  12.816366       13.00  0.567214   
9     241132000         9   416.500000  11.092090       11.10  0.206946   
10    241132000        10   450.333333  12.635598       12.90  0.794641   
11    241132000        11   163.516667  11.107299       11.10  0.208155   
12    241132000        12

In [17]:
trainIdxTanker = []
for iDX in range(trajDFTank.shape[0]):
    tempTrajInfo = trajDFTank.iloc[iDX]
    existInTest = 0
    if(tempTrajInfo['TRAJ_TIME'] > 90):
        for testIDX in range(trajDFTestTanker.shape[0]):
            tempTestTrajInfo = trajDFTestTanker.iloc[testIDX]
            
            if(tempTrajInfo['MMSI'] == tempTestTrajInfo['MMSI']) \
            and (tempTrajInfo['TRAJ_NUM'] == tempTestTrajInfo['TRAJ_NUM']):
                existInTest = 1
                break
        if(existInTest == 0):
            trainIdxTanker.append(iDX)

In [19]:
print(len(trainIdxTanker))
#contains list of trajectories used for training
trajDFTrainTanker = trajDFTank.iloc[trainIdxTanker,:].reset_index(drop=True)
print(trajDFTrainTanker)

3742
           MMSI  TRAJ_NUM    TRAJ_TIME   SOG_MEAN  SOG_MEDIAN   SOG_STD  \
0     241132000         1  1108.833333  11.416632       11.50  0.339299   
1     241132000         2  1099.183333  11.668898       11.60  0.364481   
2     241132000         3   254.500000  11.174439       11.00  0.415598   
3     241132000         4   429.850000  10.641176       10.70  0.285981   
4     241132000         6  1076.333333  11.917368       12.00  0.640760   
5     241132000         9   416.500000  11.092090       11.10  0.206946   
6     241132000        11   163.516667  11.107299       11.10  0.208155   
7     241132000        12   157.366667  11.507438       11.50  0.133645   
8     241132000        13   414.333333  12.000565       11.85  0.786176   
9     241132000        14   194.000000  11.585294       11.60  0.291022   
10    241132000        15   203.150000  11.324859       11.30  0.224755   
11    241132000        16   960.800000  11.079381       11.20  0.364686   
12    241251000     

In [20]:
destFileTrainTanker = "/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1024_Train.csv"
print(destFileTrainTanker)
aISDM.save_data_to_csv(trajDFTrainTanker, destFileTrainTanker)

/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Output/1024_Train.csv


0